- Source: https://tableau.github.io/server-client-python/docs/api-ref
- "HDC현대산업개발": "2a21240d-3f53-4f02-944d-b01647c3dd51"

In [ ]:
# hr["have_account"] = hr.apply(lambda x:True if x["num"] in all_users["user_name"].tolist() else False, axis=1)

In [1]:
import tableauserverclient as TSC
import pandas as pd
import numpy as np
import pickle as pk

pd.set_option("display.max_rows", None)

In [2]:
hr = pd.read_excel("D:/디지털혁신팀/태블로/직원명단_조직_210630.xlsx")
hr["성명"] = hr["성명"].apply(lambda x:x.replace(" ", ""))
hr = hr[["사번", "EMAIL", "본부명(조직단위기준)", "조직명", "하위그룹", "직무", "직급", "성명"]]
hr = hr.rename({"사번":"num", "EMAIL":"email", "본부명(조직단위기준)":"hq", "조직명":"dep", "하위그룹":"sub", "직무":"job", "직급":"rank", "성명":"name"}, axis=1)
hr = hr.sort_values(by=["job", "rank", "hq", "dep"], axis=0)
hr["num"] = hr["num"].map(str)

hr.loc[hr["num"] == "1", "num"] = "0001"

In [3]:
empls = {str(row["num"]):(row["hq"], row["dep"], row["job"], row["name"]) for _, row in hr.iterrows()}

req_opts = TSC.RequestOptions(pagesize=1000)

## SIgn In

In [14]:
with open("D:/Github/Work/Tableau/admin_TOKEN.pkl", "rb") as f:
    token = pk.load(f)
tableau_auth = TSC.PersonalAccessTokenAuth("admin_TOKEN", token)
server = TSC.Server("http://218.153.56.75/", use_server_version=True)

## Create Fundamental Datasets

In [62]:
# all_users.csv
with server.auth.sign_in(tableau_auth):
    all_users, pagination_item = server.users.get(req_options=req_opts)
all_users = [(user.id, user.name, user.fullname, user.site_role) + empls[user.name] if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None) for user in all_users]
all_users = pd.DataFrame(all_users, columns=["user_id", "user_name", "user_fullname", "site_role", "hq", "dep", "job", "name"])

all_users.to_csv("D:/Github/Work/Tableau/all_users.csv", index=False, encoding="euc-kr")

# all_groups.csv
with server.auth.sign_in(tableau_auth):
    all_groups, pagination_item = server.groups.get(req_options=req_opts)
all_groups = [(group.name, group.id) for group in all_groups]
all_groups = pd.DataFrame(all_groups, columns=["group", "group_id"])

all_groups.to_csv("D:/Github/Work/Tableau/all_groups.csv", index=False, encoding="euc-kr")

# user_group.csv
user_group = list()
with server.auth.sign_in(tableau_auth):
    users, pagination_item = server.users.get(req_options=req_opts)
    for user in users:
        pagination_item = server.users.populate_groups(user, req_options=req_opts)
        user_group.extend([(user.id, user.name, user.fullname, user.site_role) + empls[user.name] + (group.name,) if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None, group.name) for group in user.groups if group.name != "All Users"])
user_group = pd.DataFrame(user_group, columns=["user_id", "user_name", "user_fullname", "site_role", "hq", "dep", "job", "name", "group"])

user_group.to_csv("D:/Github/Work/Tableau/user_group.csv", index=False, encoding="euc-kr")

# group_user.csv
group_user = list()
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    for group in groups[1:]:
        pagination_item = server.groups.populate_users(group, req_options=req_opts)
        group_user.extend([(group.name,) + empls[user.name] + (user.site_role,) if user.name in empls.keys() else (group.name, None, None, None, user.name, user.site_role) for user in group.users])

group_user = pd.DataFrame(group_user, columns=["group", "hq", "dep", "job", "name", "site_role"])

group_user.to_csv("D:/Github/Work/Tableau/group_user.csv", index=False, encoding="euc-kr")

# user_workbook
user_wb = list()
with server.auth.sign_in(tableau_auth):
    users, pagination_item = server.users.get(req_options=req_opts)
    for user in users:
        pagination_item = server.users.populate_workbooks(user, req_options=req_opts)
        user_wb.extend([(user.id, user.name, user.fullname, user.site_role) + empls[user.name] + (wb.name,) if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None, wb.name) for wb in user.workbooks if wb.project_name == "HDC현대산업개발"])
user_wb = pd.DataFrame(user_wb, columns=["user_id", "user_name", "user_fullname", "site_role", "hq", "dep", "job", "name", "wb"])

user_wb.to_csv("D:/Github/Work/Tableau/user_wb.csv", index=False, encoding="euc-kr")
            
print("Completed!")

Completed!


## 그룹별 유저 배정

In [71]:
# "HR현황"
ls = hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장")]["num"].tolist()
for num in ls:
    try:
        user_id = all_users.loc[all_users["user_name"] == num, "user_id"].values[0]
    except:
        print(f"No Account; {num}")
#         with server.auth.sign_in(tableau_auth):
#             new_user = TSC.UserItem(num, "Viewer")
#             new_user = server.users.add(new_user)   
#             new_user.fullname = f"{hr.loc[hr["num"] == num, "name"].values[0]}({hr.loc[hr["num"] == num, "dep"].values[0]})"
#             new_user.email = hr.loc[hr["num"] == num, "email"].values[0]
#             new_user = server.users.update(new_user, password=f"hdc{name}")
#             user_id = all_users.loc[all_users["user_name"] == num, "user_id"].values[0]
    with server.auth.sign_in(tableau_auth):
        groups, pagination_item = server.groups.get(req_options=req_opts)
        group = [i for i in groups if i.id == "8bae0a19-c83c-4713-b932-a489754653de"][0]
        server.groups.add_user(group, user_id)

## 그룹별 유저 기준

### 현장 실적 + 예산변경 현황, CS접수현황, HR현황, 협업플랫폼 사용현황
- 사용자 전체

### 분양일보("16065e45-21b3-4e1b-80e1-c7c41079adfd")
```python
hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장") | ((hr["dep"] == "도시개발1팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "도시개발2팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "도시정비1팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "도시정비2팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "마케팅팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "민간수주1팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "민간수주2팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "고객서비스팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "경영분석팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "재무팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "회계팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "디지털플랫폼팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "미래전략팀") & (hr["job"] == "팀장")) | (hr["num"] == "6110") | (hr["num"] == "4001380")]
```
- 회장님 비서?, 상임자문?, 부문장?, 김대중?

### 수주정보
```python
hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장")]
```
- 회장님 비서?, 상임자문?

### 본부장 회의("01b579e9-e0b0-458d-b12a-94292171d3a7")
```python
hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장") | ((hr["sub"] != "정보기술외주") & (hr["job"] == "팀장")) | (hr["job"] == "실장") | (hr["job"] == "그룹장") | (hr["job"] == "부문장") | (hr["job"] == "단장") | (hr["job"] == "리더") | (hr["job"] == "PM") | (hr["job"] == "건축PM") | (hr["num"] == "6447") | (hr["num"] == "6406") | (hr["num"] == "6486") | (hr["num"] == "6427") | (hr["num"] == "6014") | (hr["num"] == "5704") | (hr["num"] == "5720") | (hr["num"] == "5882") | (hr["num"] == "4001380") | (hr["num"] == "4100435") | (hr["num"] == "6281") | (hr["num"] == "6048")]
```
- 회장님 비서?, 상임자문?, 김주완?, 성기창?, 오재환?

### 마이호미
```python
hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장") | ((hr["dep"] == "디지털플랫폼팀") & (hr["job"] == "팀장")) | (hr["num"] == "6371") | (hr["num"] == "1101296") | (hr["num"] == "6371")]
```
- 회장님 비서?, 상임자문?,

### 공수현황
```python
hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장") | ((hr["dep"] == "실시설계팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "일반견적팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "예산견적팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "디지털플랫폼팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "견적그룹(개발영업)") & (hr["job"] == "그룹장")) | ((hr["dep"] == "설계그룹(개발영업)") & (hr["job"] == "그룹장"))]
```

In [50]:
# user_workbook
with server.auth.sign_in(tableau_auth):
    user = server.users.get_by_id("b71d75c3-a34c-4dc5-a92b-de46dc681212")
    server.users.populate_workbooks(user)
    for workbook in user.workbooks:
        if workbook.project_name == "HDC현대산업개발":
            print(workbook.name)

수주정보현황
menu navi
분양일보
CS 현황
협업플랫폼 사용량
진행 현장 실적
HR현황(수정본)


## Create Users

In [8]:
hr[hr["name"]=="이윤재"]

,num,email,hq,dep,sub,job,rank,name
1024,1101291,yjlee@hdc-dvp.com,미래혁신본부,디지털플랫폼팀,계약직,팀원,계약직3급,이윤재


In [39]:
with server.auth.sign_in(tableau_auth):
    user = TSC.UserItem("1101291", "Viewer")
    user = server.users.add(user)
    print("Adding a User Completed!")

    user.fullname = "이윤재(디지털플랫폼팀)"
    user.email = "yjlee@hdc-dvp.com"
    user = server.users.update(user, password=f"hdc{user.name}")
    print("Updating a User Completed!")

Adding a User Completed!
Updating a User Completed!


## Add Users to Groups

In [73]:
# with server.auth.sign_in(tableau_auth):
#     groups, pagination_item = server.groups.get(req_options=req_opts)
#     users, pagination_item = server.users.get(req_options=req_opts)
#     for _, row in hr[hr["check"].notna()].iterrows():
#         idx = all_groups[all_groups["group"] == row["check"]].index[0]
#         group = groups[idx]
        
#         user_idx = all_users[all_users["user_name"] == row["num"]].index[0]
#         user_id = users[user_idx].id
    
#         server.groups.add_user(group, user_id)

In [41]:
keyword = "혁신팀"
num = "1101291"

group_idx = all_groups[all_groups["group"].str.contains(keyword)].index[0]
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    group = groups[group_idx]
    
    user_idx = all_users[all_users["user_name"] == num].index[0]
    users, pagination_item = server.users.get(req_options=req_opts)
    user_id = users[user_idx].id
    
    server.groups.add_user(group, user_id)

## Modify User Information

In [43]:
with server.auth.sign_in(tableau_auth):
    user = server.users.get_by_id("38fdc3c6-4b8f-4dff-8efd-29faf7e175a7")

    user.fullname = "이윤재(디지털플랫폼팀)"
    user.site_role = "ServerAdministrator"
    user = server.users.update(user)
    print("Updating a User Completed!")

In [1]:
# for _, row in all_users.iterrows():
#     if row["dep"] != None:
#         print(row["user_id"], row["user_fullname"], row["name"], row["dep"])

# with server.auth.sign_in(tableau_auth):
#     user = server.users.get_by_id(row["user_id"])
#     user.fullname = f"{row['name']}({row['dep']})"
    
#     user_rev = server.users.update(user)

## Remove Users

In [38]:
user_id = "0f4f7e47-0e69-4e2e-8f3b-74dbde7157fd"
with server.auth.sign_in(tableau_auth):
    server.users.remove(user_id)